In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import surfaceAreaEstimators as sea
import COTEstimators as ce

from pathlib import Path

# Global plot parameters
plt.rcParams.update({'font.size':40, 'lines.markersize':18})
plt.rcParams.update({'mathtext.fontset':'cm'})
plt.rcParams.update({'font.family':'Times New Roman'})

In [ ]:
def power_fit_lmfit(params, x, y):
        a = params['a']
        b = params['b']
        y_fit = a* x **b
        
        return y_fit-y

In [ ]:
dataBase_path = r'./'
dataBase_file = r'../UUVDatabase.xlsx'
linkage_file = r'Linkage_data.xlsx'

In [ ]:
data_path = Path(dataBase_path, dataBase_file)
bcfDF = pd.read_excel(data_path, sheet_name = 'BCF')
mpfDF = pd.read_excel(data_path, sheet_name='MPF')
liftDF = pd.read_excel(data_path, sheet_name="LiftBased")

In [ ]:
bcf_cot = bcfDF[bcfDF['COTopt [J/m]'].notnull()]
mpf_cot = mpfDF[mpfDF['COTopt [J/m]'].notnull()]
lift_cot = liftDF[liftDF['COTopt [J/m]'].notnull()]

# sort the column values in ascending order
bcf_cot = bcf_cot.sort_values('Weight [kg]')
mpf_cot = mpf_cot.sort_values('Weight [kg]')
lift_cot = lift_cot.sort_values('Weight [kg]')

# drop all rows with nan values
bcf_cot.dropna(subset=['Weight [kg]'], inplace=True)
mpf_cot.dropna(subset=['Weight [kg]'], inplace=True)
lift_cot.dropna(subset=['Weight [kg]'], inplace=True)

In [ ]:
# break up the data frame into perspective locomotion modes
bcf_modes = bcf_cot['Locomotion'].unique()
mpf_modes = mpf_cot['Locomotion'].unique()
lift_modes = lift_cot['Locomotion'].unique()

bcf_dfs = [bcf_cot[bcf_cot['Locomotion']==mode] for mode in bcf_modes]
mpf_dfs = [mpf_cot[mpf_cot['Locomotion']==mode] for mode in mpf_modes]
lift_dfs = [lift_cot[lift_cot['Locomotion']==mode] for mode in lift_modes]

In [ ]:
plt.rcParams.update({'font.size':40, 'lines.markersize':12})
fit_method = 'least_squares'

markers = ['o', 'v', '*', '^', 'd']


# plot COT and Number of actuators
fig1, ax1 = plt.subplots(figsize=(15,13), facecolor='w')
fig2, ax2 = plt.subplots(figsize=(15,13), facecolor='w')

for i, mode in enumerate(bcf_modes):
    ax1.scatter(bcf_dfs[i]['No Linkages'], bcf_dfs[i]['COTopt [J/kgm]'], marker=markers[i], label=mode)
    ax2.scatter(bcf_dfs[i]['No Actuators'], bcf_dfs[i]['COTopt [J/kgm]'], marker=markers[i], label=mode)

for i, mode in enumerate(mpf_modes):       
    ax1.scatter(mpf_dfs[i]['No Linkages'], mpf_dfs[i]['COTopt [J/kgm]'], marker=markers[i], label=mode)
    ax2.scatter(mpf_dfs[i]['No Actuators'], mpf_dfs[i]['COTopt [J/kgm]'], marker=markers[i], label=mode)

for i, mode in enumerate(lift_modes):
    ax1.scatter(lift_dfs[i]['No Linkages'], lift_dfs[i]['COTopt [J/kgm]'], marker=markers[i], label=mode)
    ax2.scatter(lift_dfs[i]['No Actuators'], lift_dfs[i]['COTopt [J/kgm]'], marker=markers[i], label='Lift')
    
ax1.set_xlabel('Number of Linkages')
ax1.set_ylabel('$\mathrm{COT_{opt}}$ [J/kgm]')
ax1.set_xticks(np.arange(0, 20, 2))
ax1.set_yscale('log')
#ax1.grid(which='both', linestyle='--')
ax1.legend(loc=1, ncol=2, bbox_to_anchor=(1.02, 1.46))

ax2.set_xlabel('Number of Actuators')
ax2.set_ylabel('$\mathrm{COT_{opt}}$ [J/kgm]')
ax2.set_xticks(np.arange(0, 20, 2))
ax2.set_yscale('log')
#ax2.grid(which='both', linestyle='--')
ax2.legend(loc=1, ncol=2, bbox_to_anchor=(1.02, 1.46))
fig1.savefig('uuv_cot_linkages.pdf', facecolor='w', dpi=300, bbox_inches='tight')
fig2.savefig('uuv_cot_actuators.pdf', facecolor='w', dpi=300, bbox_inches='tight')

In [ ]:
data_path = Path(dataBase_path, linkage_file)
xlsx = pd.ExcelFile(data_path)
sheets = xlsx.sheet_names

linkage_dfs = [pd.read_excel(xlsx, sheet_name=sheet) for sheet in sheets]

In [ ]:
fig3, ax3 = plt.subplots(figsize=(12,10), facecolor='w')
fig4, ax4 = plt.subplots(figsize=(12,10), facecolor='w')

for i, name in enumerate(sheets):
    if name != 'Master' and name != 'Rigid Tail' and name != 'Compliant Tail':
        linkage_dfs[i].sort_values('No Linkages', inplace=True)
        if name == 'Servo':
            dx = np.linspace(0, 10, 20)
            a, b, c = 2.40, -27.00, 80.00
            dy = a*dx**2 + b*dx + c

            ax3.scatter(linkage_dfs[i]['No Linkages'], linkage_dfs[i]['COTopt [J/kgm]'], marker='o', color='C0', label=name)
            # ax3.plot(dx, dy, 'C0--')
        elif name == 'Motor':
            data = linkage_dfs[i][linkage_dfs[i]['No Linkages'] <= 9]
            data = data[data['COTopt [J/kgm]'] < 1e3]   
            dx = np.linspace(0, 4, 20)
            a, b, c = 4.90, -36.00, 70.00
            dy = a*dx**2 + b*dx + c

            ax3.scatter(linkage_dfs[i]['No Linkages'], linkage_dfs[i]['COTopt [J/kgm]'], marker='o', color='C1', label=name)
            #ax3.scatter(linkage_dfs[i]['No Linkages'], linkage_dfs[i]['COTopt [J/kgm]'], marker='o', color='C1', s=marker_size, label=name)
            # ax3.plot(dx, dy, 'C1--')
        else:
            ax3.scatter(linkage_dfs[i]['No Linkages'], linkage_dfs[i]['COTopt [J/kgm]'], marker='o', color='C2', label=name)

for i, name in enumerate(sheets):
    if name != 'Master' and name != 'Rigid Tail' and name != 'Compliant Tail':
        linkage_dfs[i].sort_values('No Actuators', inplace=True)
        if name == 'Servo':
            low = linkage_dfs[i][linkage_dfs[i]['No Actuators'] <= 6]
            high = linkage_dfs[i][linkage_dfs[i]['No Actuators'] > 6]

            dx = np.linspace(0, 4, 20)
            a, b, c = 5.00, -35.00, 80.00
            dy = a*dx**2 + b*dx + c
            ax4.scatter(linkage_dfs[i]['No Actuators'], linkage_dfs[i]['COTopt [J/kgm]'], marker='o', color='C0', label=name)

            fit_high = np.polyfit(high['No Actuators'].values, high['COTopt [J/kgm]'].values, 1)
            poly_high = np.poly1d(fit_high)

            # ax4.plot(dx, dy, 'C0--')
            # ax4.plot(high['No Actuators'].values, poly_high(high['No Actuators'].values), 'C0--')
            # ax4.scatter([], [], color='C0', marker='o', s=50, label=name + ' (Linkages > 10): ' + str(np.round(fit_high[0],2)) + '$x$ + ' + str(np.round(fit_high[1],2)))
        elif name == 'Motor':
            data = linkage_dfs[i][linkage_dfs[i]['No Actuators'] <= 7]
            data = data[data['COTopt [J/kgm]'] < 1e3]
            
            dx = np.linspace(0, 4, 20)
            a, b, c = 5.00, -37.00, 80.00
            dy = a*dx**2 + b*dx + c

            ax4.scatter(linkage_dfs[i]['No Actuators'], linkage_dfs[i]['COTopt [J/kgm]'], marker='o', color='C1', label=name)
            # ax4.plot(dx, dy, 'C1--')
        else:
            ax4.scatter(linkage_dfs[i]['No Actuators'], linkage_dfs[i]['COTopt [J/kgm]'], marker='o', color='C2', label=name)

ax3.set_xlabel('Number of Linkages')
ax3.set_ylabel('$\mathrm{COT_{opt}}$ [J/kgm]')
ax3.set_xticks(np.arange(0, 18, 2))
#ax3.set_ylim([-10, 125])
#ax3.grid(which='both', linestyle='--')
ax3.legend()
ax3.set_yscale('log')

ax4.set_xlabel('Number of Actuators')
ax4.set_ylabel('$\mathrm{COT_{opt}}$ [J/kgm]')
ax4.set_xticks(np.arange(0, 12, 2))
#ax4.set_ylim([-10, 125])
#ax4.grid(which='both', linestyle='--')
ax4.legend()
ax4.set_yscale('log')
fig3.savefig('motor_cot_linkages.pdf', facecolor='w', dpi=300, bbox_inches='tight')
fig4.savefig('motor_cot_actuators.pdf', facecolor='w', dpi=300, bbox_inches='tight')

In [ ]:
fig5, ax5 = plt.subplots(figsize=(12,10), facecolor='w')
fig6, ax6 = plt.subplots(figsize=(12,10), facecolor='w')

for i, name in enumerate(sheets):
    if name == 'Rigid Tail':
        linkage_dfs[i].sort_values(by=['No Linkages'], inplace=True)
        low = linkage_dfs[i][linkage_dfs[i]['No Linkages'] <= 6]
        high = linkage_dfs[i][linkage_dfs[i]['No Linkages'] > 6]

        fit_low = np.polyfit(low['No Linkages'].values, low['COTopt [J/kgm]'].values, 2)
        poly_low = np.poly1d(fit_low)
        dx = np.linspace(0, 4, 20)
        dy = 5.0*dx**2 - 41.5*dx + 91

        fit_high = np.polyfit(high['No Linkages'].values, high['COTopt [J/kgm]'].values, 1)
        poly_high = np.poly1d(fit_high)

        ax5.scatter(linkage_dfs[i]['No Linkages'], linkage_dfs[i]['COTopt [J/kgm]'], marker='o', color='C0', label=name)
        # ax5.plot(dx, dy, 'C0--')
        # ax5.plot(high['No Linkages'].values, poly_high(high['No Linkages'].values), 'C0--')
    
    if name == 'Compliant Tail':
        ax5.scatter(linkage_dfs[i]['No Linkages'], linkage_dfs[i]['COTopt [J/kgm]'], marker='o', color='C1', label=name)

for i, name in enumerate(sheets):
    if name == 'Rigid Tail':
        linkage_dfs[i].sort_values(by=['No Actuators'], inplace=True)

        low = linkage_dfs[i][linkage_dfs[i]['No Actuators'] <= 6]
        high = linkage_dfs[i][linkage_dfs[i]['No Actuators'] > 6]

        fit_low = np.polyfit(low['No Actuators'].values, low['COTopt [J/kgm]'].values, 2)
        poly_low = np.poly1d(fit_low)
        dx = np.linspace(0, 4, 20)
        dy = 4.0*dx**2 - 34*dx + 80

        fit_high = np.polyfit(high['No Actuators'].values, high['COTopt [J/kgm]'].values, 1)
        poly_high = np.poly1d(fit_high)

        ax6.scatter(linkage_dfs[i]['No Actuators'], linkage_dfs[i]['COTopt [J/kgm]'], marker='o', color='C0', label=name)
        # ax6.plot(dx, dy, 'C0--')
        # ax6.plot(high['No Actuators'].values, poly_high(high['No Actuators'].values), 'C0--')
    
    if name == 'Compliant Tail':
        ax6.scatter(linkage_dfs[i]['No Actuators'], linkage_dfs[i]['COTopt [J/kgm]'], marker='o', color='C1', label=name)

ax5.set_xlabel('Number of Linkages')
ax5.set_ylabel('$\mathrm{COT_{opt}}$ [J/kgm]')
ax5.set_xticks(np.arange(0, 18, 2))
ax5.set_ylim([0, 100])
#ax5.grid(which='both', linestyle='--')
ax5.legend()
#ax5.set_yscale('log')

ax6.set_xlabel('Number of Actuators')
ax6.set_ylabel('$\mathrm{COT_{opt}}$ [J/kgm]')
ax6.set_xticks(np.arange(0, 12, 2))
ax6.set_ylim([0, 100])
#ax6.grid(which='both', linestyle='--')
ax6.legend()
#ax6.set_yscale('log')
fig5.savefig('rigid_cot_linkages.pdf', facecolor='w', dpi=300, bbox_inches='tight')
fig6.savefig('rigid_cot_actuators.pdf', facecolor='w', dpi=300, bbox_inches='tight')

In [ ]:
bcf_cot = bcfDF[bcfDF['No Linkages'].notnull()]
mpf_cot = mpfDF[mpfDF['No Linkages'].notnull()]
lift_cot = liftDF[liftDF['No Linkages'].notnull()]

# sort the column values in ascending order
bcf_cot = bcf_cot.sort_values('No Actuators')
mpf_cot = mpf_cot.sort_values('No Actuators')
lift_cot = lift_cot.sort_values('No Actuators')

# drop all rows with nan values
bcf_cot.dropna(subset=['No Actuators'], inplace=True)
mpf_cot.dropna(subset=['No Actuators'], inplace=True)
lift_cot.dropna(subset=['No Actuators'], inplace=True)

# break up the data frame into perspective locomotion modes
bcf_modes = bcf_cot['Locomotion'].unique()
mpf_modes = mpf_cot['Locomotion'].unique()
lift_modes = lift_cot['Locomotion'].unique()

bcf_modes = sorted(bcf_modes)
mpf_modes = sorted(mpf_modes)
lift_modes = sorted(lift_modes)

bcf_dfs = [bcf_cot[bcf_cot['Locomotion']==mode] for mode in bcf_modes]
mpf_dfs = [mpf_cot[mpf_cot['Locomotion']==mode] for mode in mpf_modes]
lift_dfs = [lift_cot[lift_cot['Locomotion']==mode] for mode in lift_modes]

In [ ]:
plt.rcParams.update({'font.size':25, 'lines.markersize':12})
fit_method = 'least_squares'

markers = ['o', 'v', '*', '^', 'd']
colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:purple', 'tab:red']

# plot COT and Number of actuators
fig7, ax7 = plt.subplots(figsize=(12,10), facecolor='w')

#axins = ax7.inset_axes([0.1, 0.5, 0.47, 0.47])

for i, mode in enumerate(bcf_modes):
    ax7.scatter(bcf_dfs[i]['No Actuators'], bcf_dfs[i]['No Linkages'], marker=markers[i], color=colors[i], label=mode)
    #axins.scatter(bcf_dfs[i]['No Actuators'], bcf_dfs[i]['No Linkages'], marker=markers[i], s=marker_size, color=colors[i], label=mode)


order = [0, 1, 2, 4, 3]
handles, labels = ax7.get_legend_handles_labels()
ax7.set_xticks(np.arange(0, 20, 2))
ax7.set_xlabel('Number of Actautors')
ax7.set_ylabel('Number of Linkages')
ax7.legend(loc=1, ncol=2, handles=[handles[idx] for idx in order], labels=[labels[idx] for idx in order], bbox_to_anchor=(1.01,1.24))

#ax7.indicate_inset_zoom(axins, edgecolor="black")

#axins.set_ylim([0, 10])
#axins.set_xlim([0, 10])

fig7.savefig('uuv_actuators_linkages.pdf', facecolor='w', dpi=300, bbox_inches='tight')

In [ ]:
data_path = Path(dataBase_path, dataBase_file)
convDF = pd.read_excel(data_path, sheet_name = 'Propeller')
bcfDF = pd.read_excel(data_path, sheet_name = 'BCF')
mpfDF = pd.read_excel(data_path, sheet_name='MPF')
liftDF = pd.read_excel(data_path, sheet_name="LiftBased")

conv_cot = convDF[convDF['COTopt [J/m]'].notnull()]
bcf_cot = bcfDF[bcfDF['COTopt [J/m]'].notnull()]
mpf_cot = mpfDF[mpfDF['COTopt [J/m]'].notnull()]
lift_cot = liftDF[liftDF['COTopt [J/m]'].notnull()]

# sort the column values in ascending order
conv_cot = conv_cot.sort_values('Weight [kg]')
bcf_cot = bcf_cot.sort_values('Weight [kg]')
mpf_cot = mpf_cot.sort_values('Weight [kg]')
lift_cot = lift_cot.sort_values('Weight [kg]')

# drop all rows with nan values
conv_cot.dropna(subset=['Weight [kg]'], inplace=True)
bcf_cot.dropna(subset=['Weight [kg]'], inplace=True)
mpf_cot.dropna(subset=['Weight [kg]'], inplace=True)
lift_cot.dropna(subset=['Weight [kg]'], inplace=True)

# drop all rows with nan values
conv_cot.dropna(subset=['Length [m]'], inplace=True)
bcf_cot.dropna(subset=['Length [m]'], inplace=True)
mpf_cot.dropna(subset=['Length [m]'], inplace=True)
lift_cot.dropna(subset=['Length [m]'], inplace=True)

In [ ]:
efficiency, kinVisc, fluidDensity = 0.81, 1e-6, 1027

# ------- Conv modes ---------------------------
D_s, A_s, slenderness = sea.equivalentSpheroid(conv_cot['Length [m]'].values, conv_cot['Weight [kg]'].values, fluidDensity)
conv_cot.insert(len(conv_cot.columns), 'SA', value=A_s)
conv_cot.insert(len(conv_cot.columns), 'SR', value=slenderness)

prop_power = ce.ITTC_pp(conv_cot['SR'].values, conv_cot['SA'].values, conv_cot['Length [m]'].values, conv_cot['Weight [kg]'], efficiency, kinVisc, fluidDensity, conv_cot['Uopt [m/s]'].values)
conv_cot.insert(len(conv_cot.columns), 'Pp', value=prop_power)

hotel_power = np.multiply(conv_cot['COTopt [J/m]'].values, conv_cot['Uopt [m/s]'].values) - prop_power
conv_cot.insert(len(conv_cot.columns), 'Ph', value=hotel_power)

# ------- BCF modes ---------------------------
D_s, A_s, slenderness = sea.equivalentSpheroid(bcf_cot['Length [m]'].values, bcf_cot['Weight [kg]'].values, fluidDensity)
bcf_cot.insert(len(bcf_cot.columns), 'SA', value=A_s)
bcf_cot.insert(len(bcf_cot.columns), 'SR', value=slenderness)

prop_power = ce.ITTC_pp(bcf_cot['SR'].values, bcf_cot['SA'].values, bcf_cot['Length [m]'].values, bcf_cot['Weight [kg]'], efficiency, kinVisc, fluidDensity, bcf_cot['Uopt [m/s]'].values)
bcf_cot.insert(len(bcf_cot.columns), 'Pp', value=prop_power)

hotel_power = np.multiply(bcf_cot['COTopt [J/m]'].values, bcf_cot['Uopt [m/s]'].values) - prop_power
bcf_cot.insert(len(bcf_cot.columns), 'Ph', value=hotel_power)

# ------- MPF modes ---------------------------
D_s, A_s, slenderness = sea.equivalentSpheroid(mpf_cot['Length [m]'].values, mpf_cot['Weight [kg]'].values, fluidDensity)
mpf_cot.insert(len(mpf_cot.columns), 'SA', value=A_s)
mpf_cot.insert(len(mpf_cot.columns), 'SR', value=slenderness)

prop_power = ce.ITTC_pp(mpf_cot['SR'].values, mpf_cot['SA'].values, mpf_cot['Length [m]'].values, mpf_cot['Weight [kg]'], efficiency, kinVisc, fluidDensity, mpf_cot['Uopt [m/s]'].values)
mpf_cot.insert(len(mpf_cot.columns), 'Pp', value=prop_power)

hotel_power = np.multiply(mpf_cot['COTopt [J/m]'].values, mpf_cot['Uopt [m/s]'].values) - prop_power
mpf_cot.insert(len(mpf_cot.columns), 'Ph', value=hotel_power)

# ------- Lift modes ---------------------------
D_s, A_s, slenderness = sea.equivalentSpheroid(lift_cot['Length [m]'].values, lift_cot['Weight [kg]'].values, fluidDensity)
lift_cot.insert(len(lift_cot.columns), 'SA', value=A_s)
lift_cot.insert(len(lift_cot.columns), 'SR', value=slenderness)

prop_power = ce.ITTC_pp(lift_cot['SR'].values, lift_cot['SA'].values, lift_cot['Length [m]'].values, lift_cot['Weight [kg]'], efficiency, kinVisc, fluidDensity, lift_cot['Uopt [m/s]'].values)
lift_cot.insert(len(lift_cot.columns), 'Pp', value=prop_power)

hotel_power = np.multiply(lift_cot['COTopt [J/m]'].values, lift_cot['Uopt [m/s]'].values) - prop_power
lift_cot.insert(len(lift_cot.columns), 'Ph', value=hotel_power)

In [ ]:
plt.rcParams.update({'font.size':40, 'lines.markersize':12})
fig8, ax8 = plt.subplots(figsize=(12,10), facecolor='w')
fig9, ax9 = plt.subplots(figsize=(12,10), facecolor='w')

ax8.scatter(conv_cot['Ph'].values, conv_cot['Weight [kg]'].values, label='Conventional')
ax8.scatter(bcf_cot['Ph'].values, bcf_cot['Weight [kg]'].values, label='BI BCF')
ax8.scatter(mpf_cot['Ph'].values, mpf_cot['Weight [kg]'].values, label='BI MPF')
ax8.scatter(lift_cot['Ph'].values, lift_cot['Weight [kg]'].values, label='BI Lift')

ax9.scatter(conv_cot['Ph'].values, conv_cot['Length [m]'].values, label='Conventional')
ax9.scatter(bcf_cot['Ph'].values, bcf_cot['Length [m]'].values, label='BI BCF')
ax9.scatter(mpf_cot['Ph'].values, mpf_cot['Length [m]'].values, label='BI MPF')
ax9.scatter(lift_cot['Ph'].values, lift_cot['Length [m]'].values, label='BI Lift')

ax8.set_xlabel('Displacement [kg]')
ax8.set_ylabel('Hotel Power [W]')
ax8.legend(loc=1, ncol=2, bbox_to_anchor=(1.03, 1.28))
ax8.set_yscale('log')
ax8.set_xscale('log')

ax9.set_xlabel('Length [m]')
ax9.set_ylabel('Hotel Power [W]')
ax9.legend(loc=1, ncol=2, bbox_to_anchor=(1.03, 1.28))
ax9.set_yscale('log')
ax9.set_xscale('log')

fig8.savefig(Path(r'./').joinpath('Ph_weight.pdf'), facecolor='w', dpi=300, bbox_inches='tight')
fig9.savefig(Path(r'./').joinpath('Ph_length.pdf'), facecolor='w', dpi=300, bbox_inches='tight')